In [1]:
from mysystem.read import StkDataProcessor

In [2]:
stk_processor = StkDataProcessor()
#stk_processor.preprocess_data()
#print(f"All Stock IDs: {stk_processor.stock_ids}")
print(f"Data Start Date: {stk_processor.start_date}")
print(f"Data End Date: {stk_processor.end_date}")

#stk_processor.get_stock_data(stock_codes=['000001.SZ'], start_date='2022-01-01', end_date='2022-01-31')
cumulative_returns_data = stk_processor.calculate_cumulative_returns(start_date='2022-01-01', end_date='2022-01-31')
print(cumulative_returns_data)

Data Start Date: 2020-01-02 00:00:00
Data End Date: 2022-12-30 00:00:00
           date     stk_id  cumulative_returns
4428 2022-01-28  688262.SH          -84.367720
468  2022-01-28  001227.SZ          -81.082566
4295 2022-01-28  688062.SH          -77.761114
4372 2022-01-28  688176.SH          -56.706570
4433 2022-01-28  688270.SH          -55.061084
...         ...        ...                 ...
2547 2022-01-28  301117.SZ          217.140898
3726 2022-01-28  603176.SH          271.020408
3687 2022-01-28  603102.SH          438.728585
4405 2022-01-28  688220.SH          948.688298
3358 2022-01-28  600941.SH         1443.913396

[4717 rows x 3 columns]
